In [1]:
import sys
import torch
import pyro
sys.path.append('../')
sys.path.append('../Scripts')

from Scripts.model import open_csv
from Scripts import bubble

In [2]:
reaction_type = "bubble"
reaction_dataset = "Fig4"
PATH = 'C:/Users/jlovr/CS532-project/Probabilistic-Programming/Project/'

reaction_id = "/" + reaction_type + "/" + reaction_dataset
document_name = PATH + "/dataset" + reaction_id + ".csv"

data = open_csv(document_name)

In [55]:
def from_theta_to_rate(theta, data, row, kinetic_model="ARRHENIUS"):
    if data[row][0] != '':
        docID = "Altanbonnet" + str(row)
        flurPosition = 17
        real_log_10_rate = 1 / float(data[row][5])
        [error, predicted_log_10_rate, real_log_10_rate, stuctureCounterUniLocal, half_context_biLocal] = bubble.main(real_log_10_rate, theta, data[row][1].rstrip(),data[row][2].rstrip(),data[row][3].rstrip(), (1000/ float (data[row][4] ))-273.15, float (data[row][8] ), float (data[row][9] ), 0, flurPosition, reaction_id, docID, kinetic_model)
        return predicted_log_10_rate, real_log_10_rate

In [4]:
def model(data):

    theta_dim = 15

    # priors
    # theta_mean = [13.0580, 5, 17.0580, 5,  10.0580, 1, 1.0580, -2,  13.0580, 1, 5.0580, 0,  4.0580, -2,   0.0402 ]
    theta_mean = [13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]
    theta = pyro.sample('theta', pyro.distributions.Normal(torch.tensor(theta_mean), torch.ones(theta_dim)))

    # likelihood
    len_data = 5
    for row in range(len_data):
        if row == 0:
            continue
        k, real = from_theta_to_rate(theta, data, row)
        pyro.sample('obs_'+str(row), pyro.distributions.Normal(k, 1), obs=torch.tensor(real))

    return theta

In [ ]:
def guide(data):

    theta_dim = 15

    # priors
    # theta_mean = [13.0580, 5, 17.0580, 5,  10.0580, 1, 1.0580, -2,  13.0580, 1, 5.0580, 0,  4.0580, -2,   0.0402 ]
    theta_mean = [13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]
    theta = pyro.sample('theta', pyro.distributions.Normal(torch.tensor(theta_mean), torch.ones(theta_dim)))

    # likelihood
    len_data = 5
    for row in range(len_data):
        if row == 0:
            continue
        k, real = from_theta_to_rate(theta, data, row)
        pyro.sample('obs_'+str(row), pyro.distributions.Normal(k, 1))

In [5]:
n_samples = 1000

# Importance Sampling

In [6]:
pyro.clear_param_store()
pyro.set_rng_seed(0)
# inference with importance sampling
importance = pyro.infer.Importance(model, guide=None, num_samples=n_samples)

print("doing importance sampling...")
emp_marginal = pyro.infer.EmpiricalMarginal(importance.run(data))

posterior_mean = emp_marginal.mean
posterior_std_dev = emp_marginal.variance.sqrt()

# report results
print(posterior_mean)
print(posterior_std_dev)
print("done.")

doing importance sampling...
tensor([ 1.2252e+01,  4.3570e+00,  1.3252e+01,  2.8311e+00,  1.3052e+01,
         2.8197e+00,  1.3010e+01,  2.9501e+00,  1.3044e+01,  2.9889e+00,
         1.3008e+01,  2.9946e+00,  1.3048e+01,  2.8676e+00, -5.0342e-03])
tensor([0.7942, 0.5401, 1.0240, 0.9313, 1.0652, 0.9382, 0.8982, 0.9479, 1.0362,
        1.0393, 1.0913, 1.0691, 1.0740, 1.0066, 0.9634])
done.


# MCMC

In [6]:
pyro.clear_param_store()
pyro.set_rng_seed(0)

nuts_kernel = pyro.infer.NUTS(model, jit_compile=True, )
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=n_samples)
mcmc.run(data)
samples = mcmc.get_samples()
samples_mean = samples['theta'].mean(axis=0)
samples_std_dev = samples['theta'].std(axis=0)

print(samples_mean)
print(samples_std_dev)

Warmup:   0%|          | 0/200 [00:00, ?it/s]

1.1219916962297528e-09
1.5564762750746585e-09
1.7510106089294432e-09
1.9732856067691942e-09
2.1348519471985288
2.3648244984452718
2.456132553498181
2.553848269970189
Singular matrix exception :( 
1e-10
Singular matrix exception :( 
1e-10
Singular matrix exception :( 
1e-10
Singular matrix exception :( 
1e-10


SystemError: <class 'UserWarning'> returned a result with an error set

# HMC

In [6]:
pyro.clear_param_store()
pyro.set_rng_seed(0)

hmc_kernel = pyro.infer.HMC(model, jit_compile=True, step_size=0.0855, num_steps=4)
mcmc = pyro.infer.MCMC(hmc_kernel, num_samples=n_samples, warmup_steps=100)
mcmc.run(data)
samples = mcmc.get_samples()

samples_mean = samples['theta'].mean(axis=0)
samples_std_dev = samples['theta'].std(axis=0)

print(samples_mean)
print(samples_std_dev)

Warmup:   0%|          | 0/200 [00:00, ?it/s]

1.1219916962297528e-09
1.5564762750746585e-09
1.7510106089294432e-09
1.9732856067691942e-09
2.1348519471985288
2.3648244984452718
2.456132553498181
2.553848269970189
Singular matrix exception :( 
2
Singular matrix exception :( 
2
Singular matrix exception :( 
2
Singular matrix exception :( 
2


SystemError: <class 'UserWarning'> returned a result with an error set

# SMC

In [25]:
# Create model and guide for Sequential Monte Carlo
pyro.clear_param_store()
pyro.set_rng_seed(0)


class Model(object):
    def __init__(self):
        self.theta_dim = 4

    def init(self, state, initial):
        print(initial)
        self.t = 0
        state['theta'] = pyro.sample('theta_init', pyro.distributions.Normal(initial, torch.ones(len(initial))))

    def step(self, state, y=None):
        self.t += 1
        K = link_function_1(state['theta'], y)
        k = link_function_2(K)
        state['theta'] = pyro.sample("theta_{}".format(self.t), pyro.distributions.Normal(k, 1), obs=y)

        return state['theta']

class Guide(object):
    def __init__(self, model):
        self.model = model
        self.theta_dim = 4

    def init(self, state, initial):
        print(initial)
        self.t = 0
        pyro.sample('theta_init', pyro.distributions.Normal(initial, torch.ones(len(initial))))

    def step(self, state, y=None):
        self.t += 1
        K = link_function_1(state['theta'], y)
        k = link_function_2(K)
        pyro.sample("theta_{}".format(self.t), pyro.distributions.Normal(k, 1))

In [35]:
pyro.set_rng_seed(0)

model = Model()
guide = Guide(model)

smc = pyro.infer.SMCFilter(model, guide, num_particles=100, max_plate_nesting=0)

data = torch.tensor([[0.]*100, [30.]*100, [0.]*100, [10.]*100])

smc.init(initial=torch.tensor([0.]*100))

for y in data[1:, :]:
    print(y)
    smc.step(y)

print("At final time step:")
theta = smc.get_empirical()["theta"]
print("mean: {}".format(theta.mean))
print("std: {}".format(theta.variance ** 0.5))


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
        30., 30., 30., 30., 30., 30., 30

In [19]:
data = data[1:]

In [52]:
from pyro.primitives import plate

def model(data):

    theta_dim = 15

    # priors
    theta_mean = torch.tensor([13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ])
    theta = pyro.sample('theta', pyro.distributions.Normal(theta_mean, torch.ones(theta_dim)))

    # likelihood
    len_data = 4
    with plate('data', 5, subsample_size=5) as ind:
        for row in range(len_data):
            if row == 0:
                continue
            k, real = from_theta_to_rate(theta, data, row)
            pyro.sample('obs_'+str(row), pyro.distributions.Normal(k, 1), obs=torch.tensor(real))

In [53]:
def guide(data):

    theta_dim = 15

    # priors
    theta_mean = pyro.param("theta_q", torch.tensor([13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3, 13.0580, 3,  13.0580, 3,   0.0402 ]))
    
    
    pyro.sample('theta', pyro.distributions.Normal(theta_mean, torch.ones(theta_dim)))

In [56]:
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO

# set up the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

n_steps = 5000
# do gradient steps
for step in range(n_steps):
    svi.step(data)

Singular matrix exception :( 
(10.0, 4.249008210612101)
Singular matrix exception :( 
Singular matrix exception :( 
(10.0, 4.008737143781354)
Singular matrix exception :( 
Singular matrix exception :( 
(10.0, 3.9301026087921493)
Singular matrix exception :( 


ValueError: at site "theta", invalid log_prob shape
  Expected [], actual [15]
  Try one of the following fixes:
  - enclose the batched tensor in a with pyro.plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions